**Imports**

In [ ]:
!pip install razdel

In [ ]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np

from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import spacy
import re

import pandas as pd

import spacy
from spacy import displacy
from spacy.lang.ru.examples import sentences
nlp_en = spacy.load("en_core_web_sm")

import re

from wordcloud import WordCloud

from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

In [ ]:
additional__symbols = ["...", "``", "--", "''", "’", "-=-", "—", "....", "“", "”"]

custom_punctuation = string.punctuation
for i in additional__symbols:
    custom_punctuation += i

custom_punctuation

In [ ]:
newStopWords = ["'s","'ll", "'ve", "'d", "'n't", "n't", "'m", "'re"]
for i in newStopWords:
    stop_words.add(i)

In [ ]:
stop_words

**Preprocessing**

In [ ]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        book_text = f.read()

    book_text = book_text.lower()
    tokens = word_tokenize(book_text)
    tokens = [token for token in tokens if token not in custom_punctuation]
    tokens = [token for token in tokens if token not in stop_words]

    lemmatizer = WordNetLemmatizer()
    lemmatized_words = []
    for word in tokens:
        pos = get_wordnet_pos(word)
        lemma = lemmatizer.lemmatize(word, pos=pos)
        lemmatized_words.append(lemma)
    lemmas = ' '.join(lemmatized_words)
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(lemmas)
    return lemmas

input_file = 'harry_and_prince.txt'
output_file = 'lemmas_final_harry.txt'
final = lemmatize(input_file, output_file)
print("Lemmatization is completed. Lemmatized text saved to", output_file)

Lemmatization is completed. Lemmatized text saved to lemmas_final_harry.txt


In [ ]:
final.split()[25:40]

**The most frequent tokens**

In [ ]:
with open("lemmas_final_harry.txt", encoding='utf-8') as txt:
    text = txt.read()
    frequencies_list = nltk.FreqDist(text.split())

    filtered_word_freq = dict((word, freq) for word, freq in frequencies_list.items() if not word.isdigit())

print(filtered_word_freq)

In [ ]:
frequencies_list.most_common(10)

In [ ]:
def plot_word_frequency(words,top_n=10):
    word_freq = nltk.FreqDist(words)
    labels = [element[0] for element in word_freq.most_common(top_n)]
    counts = [element[1] for element in word_freq.most_common(top_n)]
    plt.figure(figsize=(15, 5))
    plt.title("The most frequent tokens")
    plt.ylabel("Number")
    plt.xlabel("Token")
    plot = sns.barplot(x=labels, y=counts)
    return plot

In [ ]:
plot_word_frequency(text.split())

**The most frequent tokens without the most frequent words in English**

In [ ]:
verbs = [
    "want",
    "have",
    "do",
    "look",
    "think",
    "give",
    "turn",
    "say",
    "get",
    "make",
    "like",
    "seem",
    "go",
    "know",
    "come",
    "take",
    "ask",
    "see"
]

nouns = [
    "time",
    "person",
    "year",
    "way",
    "day",
    "thing",
    "man",
    "world",
    "life",
    "hand",
    "eye",
    "face"
]

adjectives = ["good", "new", "first", "last", "long", "great", "little", "own", "other", "old"]

names = [
    "harry",
    "potter",
    "hermione",
    "ron",
    "professor"
]

numerals = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']

adverbs = ['also', 'always', 'often', 'never', 'sometimes', 'very', 'well', 'here', 'there', 'now',
           'around', 'ago', 'still', 'just', 'yet', 'already', 'even', 'almost', 'though', 'back']

keys_to_delete = []
keys_to_delete.extend(adjectives)
keys_to_delete.extend(nouns)
keys_to_delete.extend(verbs)
keys_to_delete.extend(names)
keys_to_delete.extend(numerals)
keys_to_delete.extend(adverbs)


In [ ]:
def correlation_without_common(dictionary):
    for key, value in list(filtered_word_freq1.items()):
        if key in keys_to_delete:
            del filtered_word_freq1[key]

    sorted_data = sorted(filtered_word_freq1.items(), key=lambda x: x[1], reverse=True)
    labels = [element[0] for element in sorted_data[:10]]
    counts = [element[1] for element in sorted_data[:10]]
    plt.figure(figsize=(15, 5))
    plt.title("The most frequent tokens without meaningless ones")
    plt.ylabel("Number")
    plt.xlabel("Token")
    plot = sns.barplot(x=labels, y=counts)

    return plt.show()

In [ ]:
filtered_word_freq1 = filtered_word_freq

final = correlation_without_common(filtered_word_freq1)
final

**Correlation of the most frequent tokens**

In [ ]:
def correlation(input_file):
     with open(input_file, encoding='utf-8') as txt:
        filtered_tokens = txt.read()

#create a Counter object from the list of filtered tokens, which counts the frequency of each token in the text
        filtered_tokens = filtered_tokens.split()
        word_freq = Counter(filtered_tokens)
#extract the num_common_words most common tokens from the counter using the most_common method
        num_common_words = 10
        most_common_words = word_freq.most_common(num_common_words)
#initialize a matrix with dimensions num_common_words x num_common_words to store the co-occurrence counts between tokens.
        co_occurrence_matrix = np.zeros((num_common_words, num_common_words), dtype=int)
# create a dictionary word_to_index to map each token to its index in the most common words list
        word_to_index = {word: i for i, (word, _) in enumerate(most_common_words)}

#iterate through the list of tokens and update the co-occurrence matrix for consecutive token pairs.
# the co-occurrence count is incremented in the matrix.
# compute the correlation matrix using np.corrcoef based on the co-occurrence matrix.
        for i in range(len(filtered_tokens) - 1):
            if filtered_tokens[i] in word_to_index and filtered_tokens[i + 1] in word_to_index:
                index_i = word_to_index[filtered_tokens[i]]
                index_j = word_to_index[filtered_tokens[i + 1]]
                co_occurrence_matrix[index_i][index_j] += 1
                co_occurrence_matrix[index_j][index_i] += 1

        correlation_matrix = np.corrcoef(co_occurrence_matrix)
        plt.imshow(correlation_matrix, cmap='coolwarm', interpolation='nearest')
        plt.colorbar()
        plt.xticks(np.arange(num_common_words), [word for word, _ in most_common_words], rotation=45)
        plt.yticks(np.arange(num_common_words), [word for word, _ in most_common_words])
        plt.title('Correlation between Most Common Tokens')

        return plt.show()

In [ ]:
input_file = "lemmas_final_harry.txt"

final = correlation(input_file)
final

**Topic Recognition**

In [ ]:
topics = {
    "Voldemort's Return": ["voldemort", "deatheaters", "dark magic", "Snape",
                           "Malfoy", "death Eaters", "tom riddle",
                           "vanishing cabinet", "resurgence", "half-blood prince",
                           "evil", "unforgivable curses", "avada Kedavra",
                           "inferi", "sectumsempra", "nagini", "forbidden forest",
                           "dark mark", "kill", "the room of requirement"],
    "Dumbledore's Secrets and Mentorship": ["dumbledore", "mentorship", "pensieve",
                                            "memories", "memory", "past", "lesson",
                                            "headmaster", "the order of the phoenix",
                                            "thestrals", "phoenix", "prophecy", "the chosen one"],
    "Romantic Relationships": ["snog", "date", "relationships", "kiss", "love",
                               "dating", "ginny", "lavender brown", "cormac mclaggen",
                               "dean thomas"],
    "Horcruxes": ["horcruxes", "horcrux", "immortality", "soul", "destroy",
                  "diary", "ring", "lake", "cave", "nagini", "orphanage",
                  "Slughorn's memory", "Slughorn", "felix felicis", "felix"],
    "Wizarding World Politics": ["ministry of magic", "minister", "fudge",
                                 "scrimgeour", "azkaban", "muggles", "daily prophet",
                                 "quibbler", "wizengamot"],
    "Loss and Sacrifice": ["loss", "sacrifice", "death", "funeral", "tragedy",
                           "sirius", "dumbledore"],
    "Magic and Education": ["hogwarts", "magic", "lesson", "spells", "classes",
                            "professor", "student", "quidditch", "transfiguration",
                            "potions", "cauldron", "wand",
                            "defense against the dark arts", "charms", "apparition", "apparate", ],
}
with open('harry_and_prince.txt', 'r', encoding = "utf-8") as f:
    text = f.read()
    text = text.lower()
topic_freq = {topic: 0 for topic in topics}

for word in text.lower().split():
    for topic, keywords in topics.items():
        if word in keywords:
            topic_freq[topic] += 1

topics = []
numbers = []
for topic, freq in topic_freq.items():
    topics.append(topic)
    numbers.append(freq)
    # print(f"{topic}: {freq} keywords found")

plt.figure(figsize=(10, 6))
plt.bar(topics, numbers, color='skyblue')
plt.xlabel('Topics')
plt.ylabel('Keyword Counts')
plt.title('Keyword Counts for Different Topics')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

plt.show()

**Splitting into chapters**

In [ ]:
with open("harry_and_prince.txt", encoding='utf-8') as txt:
    book_text = txt.read()

chapters = re.split(r'Chapter \d+: ', book_text)[1:]

**Frequency-based sammarizer**

In [ ]:
print(f"The book has {len(chapters)} chapters")
print(f"Enter the number of the chapter you want a summary of")

number = int(input())

The book has 30 chapters
Enter the number of the chapter you want a summary of
25


In [ ]:
def summarizer(text):
    stopWords = set(stopwords.words("english"))
    words = word_tokenize(text)

    #importing stopwords from NLTK and tokenizing the input text, excluding stopwords and custom punctuation marks
    #creating a frequency table using a Counter object

    freqTable = Counter([w.lower() for w in words if w.lower() not in stopWords and w.lower() not in custom_punctuation])
    sentences = sent_tokenize(text)
    sentenceValue = dict()

# iterating through each sentence and assigning a value based on the frequency of important words present in the sentence.
# the value of each sentence is the sum of frequencies of important words it contains.
    for sentence in sentences:
        for word, freq in freqTable.items():
            if word in sentence.lower():
                if sentence in sentenceValue:
                    sentenceValue[sentence] += freq
                else:
                    sentenceValue[sentence] = freq
    # calculating the average value of sentences ased on the sum of values of all sentences
    # divided by the number of sentences.
    # selecting sentences with values greater than 1.7 times the average value.

    sumValues = 0
    for sentence in sentenceValue:
        sumValues += sentenceValue[sentence]

    average = sumValues / len(sentenceValue)
    summary = ''
    i = 0
    for sentence in sentences:
        if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.7 * average)):
            i += 1
            summary += " " + sentence

    return summary

In [ ]:
n = number - 1

final = summarizer(chapters[n])

print(final)

In [ ]:
len(final)

10745

In [ ]:
len(chapters[n])

29598

**Word Cloud**

In [ ]:
with open("lemmas_final_harry.txt", 'r', encoding='utf-8') as f:
        book_text = f.read()

In [ ]:
wordcloud = WordCloud(width = 1500,
                      height = 1000,
                      background_color='pink',
                      colormap='Pastel1').generate(book_text)
plt.figure(figsize=(40, 30))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

**KeyWord Extraction**

In [ ]:
text_data = chapters

tfidf_vectorizer = TfidfVectorizer(max_features=10, stop_words='english')

tfidf_matrix = tfidf_vectorizer.fit_transform(text_data)

feature_names = tfidf_vectorizer.get_feature_names_out()

tfidf_scores = tfidf_matrix.toarray()

top_keywords_per_document = []
for i, scores in enumerate(tfidf_scores):
    top_indices = scores.argsort()[-5:][::-1]
    top_keywords = [feature_names[idx] for idx in top_indices]
    top_keywords_per_document.append(top_keywords)

for i, keywords in enumerate(top_keywords_per_document):
    print(f"Chapter {i+1}: {', '.join(keywords)}")

**Syntaxis and children**

In [ ]:
def table_of_children(input_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        book_text = f.read()
        doc = nlp_en(book_text)
        tokens_list = []
        numbers_list = []
        children_list = []
        for token in doc:
            tokens_list.append(token)
            children = [child for child in token.children]
            children = [child for child in children if str(child)[0].isalpha()]
            numbers_list.append(len([child for child in children]))
            children_list.append(children)

        df = pd.DataFrame({
        'Token': tokens_list,
        'Number of children': numbers_list,
        "Children": children_list})

        answer = df.sort_values(by='Number of children', ascending=False)
        return answer.head(10)

input_file = 'harry_and_prince.txt'
final = table_of_children(input_file)

In [ ]:
final

**Syntactic Analysis**

In [ ]:
def syntatic_analysis(sentence):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(sentence)
    displacy.render(doc, style="dep", options={"compact": True, "distance": 100})

In [ ]:
print("Enter a sentence")
text = input()


final = syntatic_analysis(text)
final

**Morphology**

In [ ]:
def morphology(input_file):
    with open(input_file, encoding='utf-8') as f:
            book_text = f.read()
    doc = nlp_en(book_text)

    lst_tokens = []
    lst_pos = []
    lst_morph = []

    for token in doc:
        lst_tokens.append(token.text)
        lst_pos.append(token.pos_)
        lst_morph.append(token.morph)

    df = pd.DataFrame({
                'Token': lst_tokens,
                'POS': lst_pos,
                'Morphology': lst_morph}
                    )

    return df.head(10)

In [ ]:
input_file = "harry_and_prince.txt"

final = morphology(input_file)
final

**POS**

In [ ]:
def parts_of_speech(input_file):
    lst_pos = []

    with open(input_file, encoding='utf-8') as f:
            book_text = f.read()

    book_text = re.sub('[^\w ]+', '', book_text)
    doc = nlp_en(book_text.lower())

    for token in doc:
        lst_pos.append(token.pos_)

    final_pos1 = set(lst_pos)
    sum_ = len(lst_pos)
    final_pos1 = sorted(list(set(lst_pos)))
    sorted(final_pos1)

    final_count1 = []
    final_part1 = []
    for part_of_speech in final_pos1:
        final_count1.append(lst_pos.count(part_of_speech))
        final_part1.append(lst_pos.count(part_of_speech) / sum_)

    df = pd.DataFrame({
            'Part of speech': final_pos1,
            'Number': final_count1,
            "Ratio": final_part1})

    return df

In [ ]:
input_file = "harry_and_prince.txt"

final = parts_of_speech(input_file)
final

**NER**

In [ ]:
nlp = spacy.load("en_core_web_sm")
with open('harry_and_prince.txt', 'r', encoding = "utf-8") as f:
    text = f.read()

doc = nlp(text)

In [ ]:
all_entities = []
entities = set()
for ent in doc.ents:
    entities.add(ent.label_)
    all_entities.append(ent.label_)

entities = sorted(entities)

In [ ]:
Counter(entities)

Counter({'CARDINAL': 1, 'DATE': 1, 'EVENT': 1, 'FAC': 1, 'GPE': 1, 'LAW': 1, 'LOC': 1, 'NORP': 1, 'ORDINAL': 1, 'ORG': 1, 'PERSON': 1, 'PRODUCT': 1, 'QUANTITY': 1, 'TIME': 1, 'WORK_OF_ART': 1})

In [ ]:
entity_counter = Counter(all_entities)
plt.bar(entity_counter.keys(), entity_counter.values())
plt.xlabel("Entity Type")
plt.ylabel("Frequency")
plt.title("Frequency of Named Entity Types")
plt.xticks(rotation=45)
plt.show()

In [ ]:
all_names = Counter([ent.text for ent in doc.ents if ent.label_ == "PERSON"])

top_names = all_names.most_common(20)

names, name_counts = zip(*top_names)
plt.figure(figsize=(10, 6))
plt.barh(names, name_counts, color='skyblue')
plt.xlabel('Frequency')
plt.ylabel('Names')
plt.title('Top 10 Most Common Names')
plt.gca().invert_yaxis()
plt.show()

**BookNLP**

In [ ]:
#%pip install booknlp

In [ ]:
##from booknlp.booknlp import BookNLP
#import pandas as pd
#from pprint import pprint

In [ ]:
#model_params={
		#"pipeline":"entity,quote,supersense,event,coref",
		#"model":"big"
#}

#booknlp=BookNLP("en", model_params)

In [ ]:
####input_file = "harry_and_prince.txt"
###output_directory = "HarryPotter"
##book_id = "harry"

#booknlp.process(input_file, output_directory, book_id)

In [ ]:
import sys
import csv
# wide future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

csv.field_size_limit(sys.maxsize)

131072

In [ ]:
! ls HarryPotter/

harry.book  harry.book.html  harry.entities  harry.quotes  harry.supersense  harry.tokens


**Types of roles or attributes associated with characters (or entities) identified in a text**

In [ ]:
import json
with open('HarryPotter/harry.book') as f:
    data = json.load(f)

len(data["characters"])

1369

In [ ]:
#data["characters"]
data["characters"][0].keys()

dict_keys(['agent', 'patient', 'mod', 'poss', 'id', 'g', 'count', 'mentions'])



*   **Agent**: An agent is typically a character or entity that performs an action in a narrative, usually sb who is actively engaged in actions or events within the text.



*   **Patient**: A patient is an entity that undergoes an action or is affected by it.


*   **Mod**: Modifiers are words or phrases that provide additional information about other elements in a sentence. In the context of characters, modifiers may indicate attributes, qualities, or relationships associated with the character.

*   **Poss**: Possessors indicate ownership or possession of
something by a character.

**Tokens**

In [ ]:
tokens = pd.read_csv('HarryPotter/harry.tokens', delimiter="\t", engine="python", encoding='utf-8', error_bad_lines=False)


In [ ]:
tokens.head(20)

**Named Entities**

In [ ]:
entities = pd.read_csv("HarryPotter/harry.entities", delimiter="\t")
entities.head(10)

In [ ]:
quote = pd.read_csv("HarryPotter/harry.quotes", delimiter="\t", on_bad_lines='skip')
quote.head(10)

In [ ]:
hagrid_quotes_ = quote[quote['mention_phrase'] == "Hagrid"]

In [ ]:
hagrid_quotes_

**Analyzing characters**

Code source for the next cell: https://medium.com/bitgrit-data-science-publication/booknlp-analyze-your-favorite-books-with-nlp-606367aa0930

In [ ]:
import json
from collections import Counter

def proc(filename):
    with open(filename) as file:
        data=json.load(file)
    return data

def get_counter_from_dependency_list(dep_list):
    counter=Counter()
    for token in dep_list:
        term=token["w"]
        tokenGlobalIndex=token["i"]
        counter[term]+=1
    return counter


def create_character_data(data, printTop):
    character_data = {}
    for character in data["characters"]:

        agentList=character["agent"]
        patientList=character["patient"]
        possList=character["poss"]
        modList=character["mod"]

        character_id=character["id"]
        count=character["count"]

        referential_gender_distribution=referential_gender_prediction="unknown"

        if character["g"] is not None and character["g"] != "unknown":
            referential_gender_distribution=character["g"]["inference"]
            referential_gender=character["g"]["argmax"]

        mentions=character["mentions"]
        proper_mentions=mentions["proper"]
        max_proper_mention=""

        #Let's create some empty lists that we can append to.
        poss_items = []
        agent_items = []
        patient_items = []
        mod_items = []

        # just print out information about named characters
        if len(mentions["proper"]) > 0:
            max_proper_mention=mentions["proper"][0]["n"]
            for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
                poss_items.append((v,k))

            for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
                agent_items.append((v,k))

            for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
                patient_items.append((v,k))

            for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
                mod_items.append((v,k))

            # print(character_id, count, max_proper_mention, referential_gender)
            character_data[character_id] = {"id": character_id,
                                  "count": count,
                                  "max_proper_mention": max_proper_mention,
                                  "referential_gender": referential_gender,
                                  "possList": poss_items,
                                  "agentList": agent_items,
                                  "patientList": patient_items,
                                  "modList": mod_items
                                 }

    return character_data


def find_verb_usage(character_data, analysis=["agent", "patient"]):
    new_analysis = []
    for item in analysis:
        if item == "agent":
            new_analysis.append("agentList")
        elif item == "patient":
            new_analysis.append("patientList")
    main_agents = {}
    main_patients = {}
    for character in character_data:
        temp_data = character_data[character]
        for item in new_analysis:
            for verb in temp_data[item]:
                verb = verb[1].lower()
                if item == "agentList":
                    if verb not in main_agents:
                        main_agents[verb] = [(character, temp_data["max_proper_mention"])]
                    else:
                        main_agents[verb].append((character, temp_data["max_proper_mention"]))
                elif item == "patientList":
                    if verb not in main_patients:
                        main_patients[verb] = [(character, temp_data["max_proper_mention"])]
                    else:
                        main_patients[verb].append((character, temp_data["max_proper_mention"]))
    verb_usage = {"agent": main_agents,
                 "patient": main_patients}
    return verb_usage

In [ ]:
data = proc("HarryPotter/harry.book")

In [ ]:
character_data = create_character_data(data, 20)

**Data Visualisation**

In [ ]:
character_data.keys()

In [ ]:
character_data[216]["max_proper_mention"]

'Slughorn'

In [ ]:
import matplotlib.pyplot as plt
import random

def visualize_character_data(character_data, key):
    # Extract data
    data_list = character_data[key]
    counts, items = zip(*data_list)
    m = character_data["max_proper_mention"]
    # Plot
    plt.figure(figsize=(10, 6))
    plt.barh(items, counts, color='skyblue')
    plt.xlabel('Count')
    plt.ylabel(key.capitalize() if isinstance(key, str) else 'Data')
    plt.title(f'Most Common {key.capitalize()} for {m}')
    plt.gca().invert_yaxis()
    plt.show()
    print(m)

visualize_character_data(character_data[216], 'possList')
visualize_character_data(character_data[216], 'agentList')
visualize_character_data(character_data[216], 'patientList')
visualize_character_data(character_data[216], 'modList')


**Characters**

In [ ]:
for num, character in list(character_data.items())[:10]:
    print(f"{num} - {character['max_proper_mention']}")

136 - Harry
146 - Dumbledore
217 - Ron
170 - Malfoy
194 - Hermione
358 - Hermione
503 - Harry
218 - Hagrid
192 - Voldemort
221 - Snape


In [ ]:
import matplotlib.pyplot as plt

# Extract character IDs and maximum proper mentions
character_ids = []
max_proper_mentions = []
for num, character in list(character_data.items())[:10]:
    character_ids.append(num)
    max_proper_mentions.append(character['max_proper_mention'])

# Plot
plt.figure(figsize=(10, 6))
plt.bar(max_proper_mentions, character_ids, color='skyblue')
plt.xlabel('Character ID')
plt.ylabel('Maximum Proper Mention')
plt.title('Character IDs vs Maximum Proper Mentions')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
verbs = find_verb_usage(character_data)

In [ ]:
verbs.keys()

dict_keys(['agent', 'patient'])

In [ ]:
verbs["agent"]["cried"]

[(333, 'Professor Trelawney'), (502, 'Dobby')]

In [ ]:
verbs["agent"]["knew"]

[(136, 'Harry'), (146, 'Dumbledore'), (503, 'Harry'), (192, 'Voldemort'), (221, 'Snape'), (379, 'Riddle'), (216, 'Slughorn'), (226, 'Tonks'), (144, 'Sirius'), (385, 'Stan Shunpike'), (542, 'Mr. Burke'), (367, 'Dumble - dore'), (479, 'Mal - foy')]

In [ ]:
verbs["patient"]["asked"]

[(136, 'Harry'), (146, 'Dumbledore'), (217, 'Ron'), (170, 'Malfoy'), (194, 'Hermione'), (358, 'Hermione'), (503, 'Harry'), (218, 'Hagrid'), (221, 'Snape'), (379, 'Riddle'), (236, 'Ginny'), (216, 'Slughorn'), (193, 'Slughorn'), (156, 'Snape'), (251, 'Professor McGonagall'), (227, 'Lupin'), (153, 'Katie'), (174, 'Neville'), (302, 'McLaggen'), (288, 'Luna'), (157, 'Bellatrix'), (220, 'Mr. Weasley'), (333, 'Professor Trelawney'), (241, 'Bill'), (232, 'Fred'), (419, 'Mrs. Cole'), (280, 'Borgin'), (301, 'Zabini'), (476, 'Filch'), (367, 'Dumble - dore'), (275, 'Michael Corner'), (461, 'Potty')]

In [ ]:
corpus = "lemmas_final_harry.txt"

In [ ]:
with open("lemmas_final_harry.txt", encoding='utf-8') as txt:
    corpus = txt.read()